In [17]:
import pandas as pd
import json
import time
import webbrowser as wbb
import os
import shutil

In [18]:
#json file can be downloaded from https://scryfall.com/docs/api/bulk-data from the "Unique Artwork" option

#opening json file in code, may need to use full path depending on where the json file is
with open("unique-artwork-20220307221404.json", encoding='utf-8') as f:
    data = json.load(f)

In [19]:
#converting json file into Pandas dataframe
df = pd.DataFrame(data)

In [20]:
#droping null values from column image_uris as any column with that value does not have an image
df = df.dropna(subset = ['image_uris'])

In [ ]:
#Loops through the dictionary in the image_uris column to open the png uris in the default browser of your system
for item in df['image_uris']:
    time.sleep(5)#5 second wait between images to prevent IP from being blocked, but can likely be reduced to 1
    wbb.open_new(item["png"])#once the page is opened, it immediately closes and downloads the image file
    
#Loop took 1.7 days to run in current configuration, if sleep was reduced to 1 it would take apx. 8 hours

In [21]:
#creating a list of file names from the all of the downloaded image files that are in folder MTG_Images, may need longer path
filenames = []
with os.scandir('MTG_Images/') as entries:
    for entry in entries:
        filenames.append(entry.name)

In [24]:
#creating a list pf the set and collector number from the file name
imset = []
col_num = []
for file in filenames:
    imset.append(file.split('-')[0])
    if file.split('-')[1] != 'A' and file.split('-')[1] != ' ':
        col_num.append(file.split('-')[1].strip('_').strip())
    else:
        col_num.append(file.split('-')[1].strip('_').strip()+'-'+file.split('-')[2])

In [26]:
#creating a data frame with the file name, set and collector number to merge file name with the correct card name
df_file_names = pd.DataFrame(list(zip(filenames, imset, col_num)),
               columns =['File_Name', 'set', 'collector_number'])

In [ ]:
#creating a subset of the original dataframe with the two keys, set and collector_number to merge filename with name
df_name_set = df[['name', 'set', 'collector_number']]

In [28]:
#merging on keys set and collector_number
result = pd.merge(df_name_set, df_file_names, how="left", on=["set", "collector_number"])

In [32]:
#dropping null values and duplicates
result = result.dropna(subset = ['File_Name'])
result = result.drop_duplicates(subset = ['File_Name'])

In [33]:
#There is a 1% data loss with current methods
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30271 entries, 0 to 30664
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              30271 non-null  object
 1   set               30271 non-null  object
 2   collector_number  30271 non-null  object
 3   File_Name         30271 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [34]:
#Creates folder MTG_data and creates MTG_data.csv in that folder
os.makedirs('MTG_data', exist_ok=True)  
result.to_csv('MTG_data/MTG_data.csv') 

In [35]:
#moving all image files in the csv file from the MTG_Images folder to the MTG_data folder
source_folder = 'MTG_Images/'
destination_folder = 'MTG_data/'
files_to_move = result['File_Name']

for file in files_to_move:
    # construct full file path
    source = source_folder + file
    destination = destination_folder + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)

Moved: tsp-157-fury-sliver.png
Moved: zen-21-kor-outfitter.png
Moved: xln-78-siren-lookout.png
Moved: eld-35-venerable-knight.png
Moved: ugin-146-wildcall.png
Moved: m21-326-mystic-skyfish.png
Moved: jmp-59-swamp.png
Moved: pcy-80-wall-of-vipers.png
Moved: c18-36-whiptongue-hydra.png
Moved: f08-7-wall-of-roots.png
Moved: grn-63-blood-operative.png
Moved: cns-40-selvala-s-enforcer.png
Moved: nph-109-fresh-meat.png
Moved: gtc-244-orzhov-guildgate.png
Moved: dom-10-charge.png
Moved: eld-255-island.png
Moved: m19-30-novice-knight.png
Moved: vis-59-fallen-askari.png
Moved: ody-115-afflict.png
Moved: isd-60-invisible-stalker.png
Moved: mh2-192-dakkon-shadow-slayer.png
Moved: arn-10-shahrazad.png
Moved: aer-78-destructive-tampering.png
Moved: c20-173-harmonize.png
Moved: mir-132-nocturnal-raid.png
Moved: mbs-41-flensermite.png
Moved: tvow-11-wolf.png
Moved: dtk-239-gate-smasher.png
Moved: ogw-126-world-breaker.png
Moved: p02-77-lurking-nightstalker.png
Moved: ddm-26-remand.png
Moved: uds-29-b

Moved: 9ed-144-megrim.png
Moved: znr-173-tuktuk-rubblefort.png
Moved: rtr-174-jarad-golgari-lich-lord.png
Moved: ala-17-knight-captain-of-eos.png
Moved: mrd-134-tooth-and-nail.png
Moved: cmr-16-court-of-grace.png
Moved: khm-176-icehide-troll.png
Moved: dom-16-evra-halcyon-witness.png
Moved: pvan-201-barrin.png
Moved: 7ed-99-steal-artifact.png
Moved: frf-105-hungering-yeti.png
Moved: sta-58-weather-the-storm.png
Moved: m19-249-detection-tower.png
Moved: khm-24-righteous-valkyrie.png
Moved: aer-11-consulate-crackdown.png
Moved: stx-105-hall-monitor.png
Moved: m11-247-forest.png
Moved: ktk-206-surrak-dragonclaw.png
Moved: roe-31-kor-spiritdancer.png
Moved: thb-111-pharika-s-libation.png
Moved: all-86b-deadly-insect.png
Moved: mor-117-cream-of-the-crop.png
Moved: 8ed-54-tundra-wolves.png
Moved: eld-304-rowan-fearless-sparkmage.png
Moved: sld-288-gruul-signet.png
Moved: war-48-erratic-visionary.png
Moved: wwk-71-akoum-battlesinger.png
Moved: inv-43-teferi-s-care.png
Moved: soi-165-hulking-d

Moved: sok-62-choice-of-damnations.png
Moved: scg-12-eternal-dragon.png
Moved: mmq-65-chameleon-spirit.png
Moved: mmq-116-black-market.png
Moved: dst-78-nourish.png
Moved: mmq-271-snake-pit.png
Moved: znr-94-bloodchief-s-thirst.png
Moved: m10-122-xathrid-demon.png
Moved: ugl-54-cardboard-carapace.png
Moved: som-133-wing-puncture.png
Moved: usg-33-redeem.png
Moved: dst-31-reshape.png
Moved: twwk-1-soldier-ally.png
Moved: vis-28-chronatog.png
Moved: ala-139-mosstodon.png
Moved: trtr-1-bird.png
Moved: c20-16-tayam-luminous-enigma.png
Moved: wth-32-ancestral-knowledge.png
Moved: y22-33-sap-vitality.png
Moved: thb-144-nyxborn-brute.png
Moved: ust-41a-novellamental.png
Moved: dka-103-scorch-the-fields.png
Moved: rav-273-terrarion.png
Moved: htr19-2-keeper-of-the-secret-lair.png
Moved: ulg-7-erase.png
Moved: pvan-403-mishra.png
Moved: gtc-102-mugging.png
Moved: scg-24-trap-digger.png
Moved: frf-73-hooded-assassin.png
Moved: usg-230-acridian.png
Moved: mbs-78-creeping-corrosion.png
Moved: rix-

Moved: ons-13-circle-of-solace.png
Moved: mmq-307-monkey-cage.png
Moved: ori-237-ramroller.png
Moved: iko-69-thieving-otter.png
Moved: all-10b-martyrdom.png
Moved: lrw-187-nova-chaser.png
Moved: mma-138-tribal-flames.png
Moved: hou-191-plains.png
Moved: dom-63-rescue.png
Moved: nph-127-vorinclex-voice-of-hunger.png
Moved: fut-60-spin-into-myth.png
Moved: eld-145-syr-carah-the-bold.png
Moved: bbd-84-sea-of-clouds.png
Moved: khm-48-bind-the-monster.png
Moved: roe-176-awakening-zone.png
Moved: gtc-170-gruul-ragebeast.png
Moved: tor-93-crackling-club.png
Moved: m19-203-thorn-lieutenant.png
Moved: ody-39-pianna-nomad-captain.png
Moved: mh1-82-changeling-outcast.png
Moved: c17-108-damnable-pact.png
Moved: soi-231-stoic-builder.png
Moved: ktk-242-scoured-barrens.png
Moved: ice-186-game-of-chaos.png
Moved: emn-4-drownyard-behemoth.png
Moved: rtr-131-oak-street-innkeeper.png
Moved: olgc-2018A-plateau.png
Moved: tema-1-spirit.png
Moved: mmq-6-charm-peddler.png
Moved: palp-2-island.png
Moved: hou

Moved: m15-172-chord-of-calling.png
Moved: exo-84-furnace-brood.png
Moved: kld-33-trusty-companion.png
Moved: soi-205-fork-in-the-road.png
Moved: ori-30-sentinel-of-the-eternal-watch.png
Moved: l14-1-soldier.png
Moved: dgm-43-maze-behemoth.png
Moved: cn2-17-noble-banneret.png
Moved: pls-44-lord-of-the-undead.png
Moved: isd-31-silverchase-fox.png
Moved: p02-147-sylvan-yeti.png
Moved: mic-6-curse-of-conformity.png
Moved: tvow-18-treasure.png
Moved: lrw-224-kithkin-mourncaller.png
Moved: eld-242-castle-vantress.png
Moved: sld-143-rick-steadfast-leader.png
Moved: m21-318-scavenging-ooze.png
Moved: rna-268-dovin-s-automaton.png
Moved: ons-264-gigapede.png
Moved: rix-171-siegehorn-ceratops.png
Moved: mir-278-reparations.png
Moved: p02-93-cunning-giant.png
Moved: akh-175-manglehorn.png
Moved: c20-55-shiny-impetus.png
Moved: all-23b-awesome-presence.png
Moved: mh1-124-force-of-rage.png
Moved: tmed-W3-garruk-apex-predator-emblem.png
Moved: ons-62-airborne-aid.png
Moved: jou-51-scourge-of-fleets

...

Moved: psus-12-sakura-tribe-elder.png
Moved: ust-45-spell-suck.png
Moved: sld-68-heliod-god-of-the-sun.png
Moved: p02-90-vampiric-spirit.png
Moved: con-114-knotvine-mystic.png
Moved: som-188-nim-deathmantle.png
Moved: arb-131-arsenal-thresher.png
Moved: roe-84-renegade-doppelganger.png
Moved: znr-73-risen-riptide.png
Moved: sok-20-moonwing-moth.png
Moved: sok-129-endless-swarm.png
Moved: tm21-11-cat.png
Moved: bbd-77-rushblade-commander.png
Moved: neo-30-mothrider-patrol.png
Moved: 5dn-66-goblin-brawler.png
Moved: iko-130-pyroceratops.png
Moved: znr-28-mesa-lynx.png
Moved: eld-224-jousting-dummy.png
Moved: ons-59-weathered-wayfarer.png
Moved: tlrw-1-avatar.png
Moved: neo-341-inkrise-infiltrator.png
Moved: shm-2-armored-ascension.png
Moved: rix-188-foul-orchard.png
Moved: khm-101-karfell-kennel-master.png
Moved: dom-278-pyromantic-pilgrim.png
Moved: sld-248-teferi-hero-of-dominaria.png
Moved: usg-61-attunement.png
Moved: ody-53-spiritualize.png
Moved: m14-61-jace-s-mindseeker.png
Moved:

Moved: c19-36-voice-of-many.png
Moved: gpt-152-izzet-signet.png
Moved: usg-238-blanchwood-treefolk.png
Moved: eld-105-smitten-swordmaster.png
Moved: mh1-108-sling-gang-lieutenant.png
Moved: vow-399-island.png
Moved: trna-9-sphinx.png
Moved: m12-2-alabaster-mage.png
Moved: plc-79-roiling-horror.png
Moved: dom-112-windgrace-acolyte.png
Moved: 10e-4-angelic-chorus.png
Moved: dtk-62-mirror-mockery.png
Moved: 5ed-332-titania-s-song.png
Moved: ktk-171-crackling-doom.png
Moved: uds-107-goliath-beetle.png
Moved: c18-54-coveted-jewel.png
Moved: xana-19-titanic-pelagosaur.png
Moved: p02-101-goblin-mountaineer.png
Moved: f12-3-tectonic-edge.png
Moved: sta-53-krosan-grip.png
Moved: stx-71-flunk.png
Moved: inv-337-island.png
Moved: soi-90-stitchwing-skaab.png
Moved: mp2-47-boil.png
Moved: cmd-240-zedruu-the-greathearted.png
Moved: grn-116-smelt-ward-minotaur.png
Moved: shm-17-pale-wayfarer.png
Moved: leg-29-osai-vultures.png
Moved: drk-31-mana-vortex.png
Moved: war-169-nissa-who-shakes-the-world.pn

Moved: ths-58-ordeal-of-thassa.png
Moved: psoi-242-anguished-unmaking.png
Moved: iko-195-narset-of-the-ancient-way.png
Moved: vow-18-heron-of-hope.png
Moved: ema-26-seal-of-cleansing.png
Moved: cst-381-forest.png
Moved: mid-123-stromkirk-bloodthief.png
Moved: rtr-121-death-s-presence.png
Moved: vis-4-eye-of-singularity.png
Moved: exp-19-wooded-foothills.png
Moved: ody-99-rites-of-refusal.png
Moved: lrw-261-springleaf-drum.png
Moved: plny-1-treasure.png
Moved: usg-52-songstitcher.png
Moved: rix-28-temple-altisaur.png
Moved: avr-47-deadeye-navigator.png
Moved: tmp-145-mindwhip-sliver.png
Moved: stx-115-storm-kiln-artist.png
Moved: sth-102-burgeoning.png
Moved: mrd-180-gold-myr.png
Moved: gpt-95-silhana-starfletcher.png
Moved: j21-20-managorger-phoenix.png
Moved: tmid-10-ooze.png
Moved: plc-88-melancholy.png
Moved: tm19-16-tezzeret-artifice-master-emblem.png
Moved: mrd-200-lodestone-myr.png
Moved: iko-54-gust-of-wind.png
Moved: neo-267-dismal-backwater.png
Moved: neo-42-the-wandering-empe

Moved: 5ed-206-xenic-poltergeist.png
Moved: fjmp-45-witchcraft.png
Moved: sld-253-karn-the-great-creator.png
Moved: unh-2-awol.png
Moved: fem-34b-basal-thrull.png
Moved: nem-2-avenger-en-dal.png
Moved: usg-163s-unworthy-dead.png
Moved: mrd-137-turn-to-dust.png
Moved: dtk-37-shieldhide-dragon.png
Moved: chk-169-godo-bandit-warlord.png
Moved: roe-85-sea-gate-oracle.png
Moved: tsp-19-fortify.png
Moved: emn-125-deranged-whelp.png
Moved: parl-2-island.png
Moved: tsp-279-terramorphic-expanse.png
Moved: troe-2-elemental.png
Moved: znr-25-makindi-ox.png
Moved: pcy-136-chimeric-idol.png
Moved: afr-290-iymrith-desert-doom.png
Moved: lrw-52-aquitect-s-will.png
Moved: soi-24-inquisitor-s-ox.png
Moved: 10e-317-colossus-of-sardia.png
Moved: m12-43-amphin-cutthroat.png
Moved: ons-346-mountain.png
Moved: dka-153-warden-of-the-wall.png
Moved: unh-133-world-bottling-kit.png
Moved: lgn-111-skirk-alarmist.png
Moved: war-154-bloom-hulk.png
Moved: avr-190-rain-of-thorns.png
Moved: ody-257-new-frontiers.png


Moved: iko-10-drannith-healer.png
Moved: afr-116-ray-of-enfeeblement.png
Moved: eve-168-fang-skulkin.png
Moved: sta-118-primal-command.png
Moved: rix-182-silent-gravestone.png
Moved: eve-180-twilight-mire.png
Moved: rtr-246-selesnya-guildgate.png
Moved: ice-306-adarkar-sentinel.png
Moved: inv-77-sway-of-illusion.png
Moved: pmoa-56-rumbling-slum-avatar.png
Moved: 7ed-335-island.png
Moved: aer-118-natural-obsolescence.png
Moved: 5ed-173-leshrac-s-rite.png
Moved: lrw-253-wydwen-the-biting-gale.png
Moved: fut-93-yixlid-jailer.png
Moved: dst-81-reap-and-sow.png
Moved: mps-28-sword-of-feast-and-famine.png
Moved: usg-293-dragon-blood.png
Moved: m20-80-winged-words.png
Moved: lea-285-underground-sea.png
Moved: dgm-139-golgari-cluestone.png
Moved: leg-45-anti-magic-aura.png
Moved: gtc-25-smite.png
Moved: ptk-114-independent-troops.png
Moved: dgm-122-armed.png
Moved: jmp-1-blessed-sanctuary.png
Moved: kld-110-chandra-torch-of-defiance.png
Moved: cmd-311-mountain.png
Moved: bfz-50-stasis-snare.pn